<a href="https://colab.research.google.com/github/satyamverma95/DL_NLP/blob/main/DL_NLP_Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Making sure you have all the necessary libraries installed.

In [3]:
#!pip install transformers datasets evaluate
#!pip install huggingface_hub
#!pip install datasets
#!pip install transformers==4.28.0
#!pip install --upgrade accelerate

 login to your Hugging Face account

In [4]:
from huggingface_hub import notebook_login

notebook_login()

Loading SQuAD dataset from Datasets Library.

In [5]:
from datasets import load_dataset

#Loading smaller subets of data to make sure that wholepipe line works before training with whole data.
squad = load_dataset("squad", split="train")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Dataset squad downloaded and prepared to /root/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453. Subsequent calls will reuse this data.


Spliting the dataset’s train split into a train and test set with the train_test_split method:

In [6]:
squad = squad.train_test_split(test_size=0.2)

Taking a look at the example.

In [7]:
squad["train"][496]

{'id': '5733f875d058e614000b66bc',
 'title': 'Premier_League',
 'context': 'At the inception of the Premier League in 1992–93, just eleven players named in the starting line-ups for the first round of matches hailed from outside of the United Kingdom or Ireland. By 2000–01, the number of foreign players participating in the Premier League was 36 per cent of the total. In the 2004–05 season the figure had increased to 45 per cent. On 26 December 1999, Chelsea became the first Premier League side to field an entirely foreign starting line-up, and on 14 February 2005 Arsenal were the first to name a completely foreign 16-man squad for a match. By 2009, under 40% of the players in the Premier League were English.',
 'question': 'By 2000-01, what percentage of players in the Premier League were from outside the UK and Ireland?',
 'answers': {'text': ['36'], 'answer_start': [269]}}

In [8]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [9]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]] ##This line creates a list of questions by stripping any leading or trailing whitespace from each question in the dataset.
    
    '''
    This line tokenizes the questions and the context using a tokenizer. 
    It takes the questions, examples["context"], and other specified parameters as inputs. 
    The max_length parameter sets the maximum length of the tokenized inputs, 
    truncation determines how to handle text that exceeds the maximum length 
    (in this case, only truncating the context), return_offsets_mapping=True instructs the 
    tokenizer to return the mapping between the tokens and the original character offsets, 
    and padding="max_length" pads the inputs to the maximum length.

    '''
    
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    '''
    This line retrieves the offset_mapping from the tokenizer's output and removes it from the inputs dictionary. 
    The offset mapping is a list that maps each token to its corresponding character offsets in the original text.
    '''

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]   #Retrieves the answer for the current example.
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        '''
        This line retrieves the sequence IDs of the tokenized inputs for the current example. 
        The sequence IDs indicate which part of the input each token belongs to (0 for question, 1 for context).
        '''

        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1


        '''
        If the answer is not fully inside the context, 
        it means it is outside of the considered portion of the text, 
        and (0, 0) is appended to start_positions and end_positions to label it as such.
        '''

        '''
          start_positions and end_positions are updated with the found token positions.

          Finally, start_positions and end_positions are added to the inputs dictionary as additional fields.

          The inputs dictionary, containing the tokenized inputs and the start and end positions of the answer span, is returned
        '''


        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [10]:
tokenized_squad = squad.map(preprocess_function, batched=True, remove_columns=squad["train"].column_names)

Map:   0%|          | 0/70079 [00:00<?, ? examples/s]

Map:   0%|          | 0/17520 [00:00<?, ? examples/s]

In [11]:
tokenized_squad["train"]

Dataset({
    features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
    num_rows: 70079
})

In [12]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [13]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained("distilbert-base-uncased")

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this mode

Three Steps Performed Below :

1) Defining training hyperparameters in TrainingArguments.</br>
2) Setting training arguments to Trainer along with the model, dataset, tokenizer, and data collator. </br>
3) Call train() to finetune your model. </br>

In [14]:
#!pip install --upgrade accelerate
#! pip install transformers==4.28.0

In [17]:
training_args = TrainingArguments(
    output_dir="Distil_BERT_1",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_squad["train"],
    eval_dataset=tokenized_squad["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

Cloning https://huggingface.co/satyamverma/Distil_BERT_1 into local empty directory.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,1.283600,1.136945
2,0.933300,1.100990


Epoch,Training Loss,Validation Loss
1,1.283600,1.136945
2,0.933300,1.100990
3,0.760400,1.122885


TrainOutput(global_step=13140, training_loss=1.039179128707816, metrics={'train_runtime': 9087.1571, 'train_samples_per_second': 23.136, 'train_steps_per_second': 1.446, 'total_flos': 2.060108684357069e+16, 'train_loss': 1.039179128707816, 'epoch': 3.0})

In [18]:
trainer.push_to_hub()

Upload file pytorch_model.bin:   0%|          | 1.00/253M [00:00<?, ?B/s]

Upload file runs/May20_17-18-26_a58ca788def9/events.out.tfevents.1684603111.a58ca788def9.801.2:   0%|         …

To https://huggingface.co/satyamverma/Distil_BERT_1
   0647515..6ae1e21  main -> main

   0647515..6ae1e21  main -> main

To https://huggingface.co/satyamverma/Distil_BERT_1
   6ae1e21..d0f6812  main -> main

   6ae1e21..d0f6812  main -> main



'https://huggingface.co/satyamverma/Distil_BERT_1/commit/6ae1e21511b7a22c0297cdf4354cec15b6965585'

Inference 

In [ ]:
question = "How many programming languages does BLOOM support?"
context = "BLOOM has 176 billion parameters and can generate text in 46 languages natural languages and 13 programming languages."

In [20]:
from transformers import pipeline

question_answerer = pipeline("question-answering", model="my_awesome_qa_model")
question_answerer(question=question, context=context)

{'score': 0.9206639528274536, 'start': 93, 'end': 95, 'answer': '13'}